In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import random

In [2]:
# Read corpus from file
with open('wikipedia.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

# Convert to list of sentences (if needed)
from nltk.tokenize import sent_tokenize
import nltk
# nltk.download('punkt')  # ensure you have this

corpus = sent_tokenize(raw_text)


In [4]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [5]:
def preprocess(Corpus):
    token=[]
    for sentence in corpus:
        l=[]
        sentence=sentence.lower()
        for w in sentence.split():
            w=lemmatizer.lemmatize(w)
            if w.isalpha():# and w not in stop_words:
                l.append(w)
        token.append(l)
    return token
                

In [9]:
tokens=(preprocess(corpus))
vocab = set( j for i in tokens for j in i)
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(vocab)
print(vocab)

{'extreme', 'three', 'digest', 'painful', 'established', 'gentle', 'small', 'release', 'speed', 'clean', 'million', 'case', 'causing', 'every', 'submerges', 'female', 'document', 'lower', 'or', 'africa', 'migrates', 'source', 'type', 'by', 'for', 'thus', 'campaign', 'south', 'break', 'previously', 'blister', 'week', 'eradication', 'stomach', 'in', 'allows', 'filtering', 'site', 'reside', 'if', 'centimeter', 'few', 'mammal', 'which', 'spread', 'had', 'due', 'severe', 'worm', 'inside', 'a', 'reptile', 'discourage', 'is', 'escape', 'though', 'becomes', 'parasitic', 'stick', 'began', 'dracunculiasis', 'worldwide', 'eradicate', 'when', 'occurring', 'that', 'the', 'center', 'will', 'falling', 'wrapping', 'across', 'leg', 'on', 'physician', 'widespread', 'have', 'specie', 'usually', 'relieve', 'several', 'surviving', 'program', 'exit', 'tension', 'intensely', 'with', 'medinensis', 'become', 'take', 'eradicated', 'wound', 'access', 'other', 'remove', 'from', 'there', 'second', 'endemic', 'cons

In [30]:
def generate_train(tokens,windowsize=2):
    data=[]
    for i in tokens:
        for idx,target in enumerate(i):
            context=[]
            for j in range(max(0,idx-windowsize),min(idx+windowsize+1,len(i))):
                if j!=idx:
                    context.append(i[j])
            data.append([context,target])
        
    return data         

In [32]:
data=generate_train(tokens)
print(data)

[[['called', 'is'], 'also'], [['also', 'is', 'a'], 'called'], [['also', 'called', 'a', 'parasitic'], 'is'], [['called', 'is', 'parasitic', 'infection'], 'a'], [['is', 'a', 'infection', 'by'], 'parasitic'], [['a', 'parasitic', 'by', 'the'], 'infection'], [['parasitic', 'infection', 'the', 'guinea'], 'by'], [['infection', 'by', 'guinea', 'dracunculus'], 'the'], [['by', 'the', 'dracunculus'], 'guinea'], [['the', 'guinea'], 'dracunculus'], [['person', 'becomes'], 'a'], [['a', 'becomes', 'infected'], 'person'], [['a', 'person', 'infected', 'by'], 'becomes'], [['person', 'becomes', 'by', 'drinking'], 'infected'], [['becomes', 'infected', 'drinking', 'water'], 'by'], [['infected', 'by', 'water', 'contaminated'], 'drinking'], [['by', 'drinking', 'contaminated', 'with'], 'water'], [['drinking', 'water', 'with', 'larva'], 'contaminated'], [['water', 'contaminated', 'larva', 'that'], 'with'], [['contaminated', 'with', 'that', 'reside'], 'larva'], [['with', 'larva', 'reside', 'inside'], 'that'], [

In [20]:
def one_hot_vector(word):
    vec = np.zeros(vocab_size)
    vec[word2idx[word]] = 1
    return vec


In [21]:
embedding_dim = 10  
W1 = np.random.randn(vocab_size, embedding_dim)
W2 = np.random.randn(embedding_dim, vocab_size) 

In [ ]:
[['dracunculiasis', 'will', 'the', 'second'], 'become']

In [ ]:
def forward_pass(context_words):
    embedding=[one_hot_vector(word2idx[i])for i in context_words]
    x = np.mean([one_hot_vector(w) for w in embedding], axis=0)  # Avg context
    h = np.dot(x, W1)  
    u = np.dot(h, W2)  
    y_pred = softmax(u)
    return x, h, u, y_pred


In [33]:
def backprop(x, h, y_pred, target_word, learning_rate=0.01):
    y_true = one_hot_vector(target_word)
    error = y_pred - y_true
    dW2 = np.outer(h, error)
    dW1 = np.outer(x, np.dot(W2, error))
    
    global W1, W2
    W1 -= learning_rate * dW1
    W2 -= learning_rate * dW2


SyntaxError: name 'W2' is used prior to global declaration (3290762684.py, line 7)

In [ ]:
for epoch in range(1000):
    loss = 0
    for context, target in data:
        x, h, u, y_pred = forward_pass(context)
        backprop(x, h, y_pred, target)
        loss += -np.log(y_pred[word2idx[target]] + 1e-7)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


SyntaxError: invalid syntax (3256032223.py, line 3)